# urls
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html


https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html
    
https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html

In [1]:
urls = [' https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html']

# Import

In [2]:
import requests
from functools import reduce
import csv
import os
from bs4 import BeautifulSoup
import random
import time
import re
from lxml import etree
import lxml.html
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

# Extract movie links

In [3]:
movie_links = []
for url in urls:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a',href=True):
            movie_links.append(link['href'])
    except Exception as ex:
        print('error is ', ex)

In [4]:
len(movie_links)

30000

In [5]:
movie_links[:5]

['https://en.wikipedia.org/wiki/Z.P.G.',
 'https://en.wikipedia.org/wiki/Zee_and_Co.',
 'https://en.wikipedia.org/wiki/And_Now_the_Screaming_Starts!',
 'https://en.wikipedia.org/wiki/The_Asphyx',
 'https://en.wikipedia.org/wiki/Assassin_(1973_film)']

# Stopwords

In [6]:
tokens_without_sw_intro = []
def rm_stopwords(text):
    new_text = text.lower()
    global tokens_without_sw_intro
    stop_words = set(stopwords.words('english')) 
    text_tokens_intro = word_tokenize(new_text)
    tokens_without_sw_intro = [word for word in text_tokens_intro if not word in stopwords.words()]
   
    return tokens_without_sw_intro

# Punctuation

In [7]:
new_words_intro =[]
def rm_punctuation ():
    global new_words_intro
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    punc_intro = ",".join(tokens_without_sw_intro)
    new_words_intro = tokenizer.tokenize(punc_intro)
    return new_words_intro

# Stemming

In [8]:
# streaming_intro = []
def stemming():
    global streaming_intro
    streaming_intro = []
    ps = PorterStemmer()
    for x in new_words_intro:
        
        streaming_intro.append(ps.stem(x))
    return streaming_intro

# Final normalize

In [9]:
# final_intro = []
def Final_normalize():
    global streaming_intro
    final_intro = []
    for i in streaming_intro:
        if len(i) > 3:
            final_intro.append(i)
    return " ".join(final_intro)

# extract title,intro,plot,table_info and save as tsv

In [10]:
check_list = ['Directed by','Produced by','Written by','Starring','Music by','Release date','Running time','Countries',
              'Language','Budget','Country']
vocabulary = []
# vocabulary = list(set(vocabulary)
for number,t in enumerate(movie_links[:5]):  
    web_response = requests.get(t)
    soup = BeautifulSoup(web_response.text, 'html.parser')
    title = soup.title.text.split('-')
    title = title[0]

    # start intro
    response = requests.get(t) 
    soup = BeautifulSoup(response.text, 'html.parser')
    intro =  soup.find('div' , { "id" : "toc" }).find_all_previous('p')[::-1]
    l=[]
    for i in intro:
        x = i.text.strip()
        if x != '':
            l.append(x)
    y= " ".join(l)
    # remove stopwords
    rm_stopword_intro = rm_stopwords(y)
    #remove punchtuation
    rm_punc_intro=rm_punctuation()
    # stemmer
    steming_intro = stemming()
    # final_normalize
    final_norm_intro = Final_normalize()
    # vocabulary
    vocab = list(set(final_norm_intro.split()))
    vocabulary.extend(vocab)


    # start plot
    plot =  soup.find('span' , { "class" : "mw-editsection-bracket" }).find_next('p')
    final_plot = plot.text.strip()
    # remove stopwords
    rm_stopword = rm_stopwords(final_plot)

    # remove punch
    rm_punc=rm_punctuation()

    # stemmer
    steming = stemming()

    #final_normalize
    final_norm_plot = Final_normalize()

    #table
    table = soup.find('table', class_ = 'infobox vevent')
    d={}
    for x in table.find_all('tr')[2:]:
        if x.th.text.lower() in [l.lower() for l in check_list]:
            for i in x.find_all('th'):
                for y in x.find_all('td'):
                    d[i.text]=y.text.lower()
                    for i in check_list:
                        if i not in d.keys():
                            d[i]= 'NA'

    if d['Countries'] == 'NA' and d['Country'] == 'NA':
        del d['Country']
    if d['Countries'] !='NA' and d['Country'] == 'NA':
        del d['Country']
    if d['Countries'] =='NA' and d['Country'] != 'NA':
        del d['Countries']

    w = d.values()
    d_value = "    ".join(list(w))
    # start save to tsv
 
    name = 'C:\\Users\\Profosor\\Desktop\\tsv\\article_{}.tsv'.format(number)
    with open(name, 'wt', encoding='UTF-8') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow([title])
        tsv_writer.writerow([final_norm_intro])
        tsv_writer.writerow([final_norm_plot])
        tsv_writer.writerow([d_value])


In [11]:
print(vocabulary)

['children', 'scienc', '1972', 'zero', 'campu', 'almost', 'fiction', 'inspir', 'overpopul', 'bomb', 'popul', 'film', 'whose', 'danish', 'oliv', 'chaplin', 'design', 'ehrlich', 'concern', 'reflect', 'direct', 'bound', 'michael', 'best', 'earth', 'entir', 'american', 'featur', 'paul', 'short', 'dystopian', 'growth', 'sell', 'govern', 'denmark', 'geraldin', 'year', 'star', 'oppress', 'book', 'futur', 'reed', 'bleak', 'violat', 'world', 'execut', 'edna', 'marriag', 'shot', '1972', 'pictur', 'woman', 'song', 'seven', 'susannah', 'screenplay', 'hutton', 'releas', 'mullin', 'theme', 'film', 'whose', 'london', 'british', 'upon', 'brien', 'separ', 'fool', 'last', 'design', 'singl', 'concern', 'taylor', 'direct', 'elizabeth', 'michael', 'middl', 'bicker', 'side', 'three', 'locat', 'director', 'need', 'known', 'columbia', 'album', 'cover', 'drama', 'cain', 'come', 'circl', 'well', 'famili', 'citat', 'coupl', 'base', 'brian', 'star', 'shepperton', 'novel', 'studio', 'night', 'compani', 'york', 'fo

# Inverted index

In [12]:
def createDictionary(directory):

    wordsAdded = {}

    cwd = os.getcwd()

    os.chdir(directory)
    fileList = os.listdir(directory)

    for file in fileList:

        with open(file, 'r') as f:
            words = set(map(lambda x: x[:-1] if x[-1] in [',', '!', '?', '.'] else x, f.read().lower().split()))

            for word in words:
                for i,term in enumerate(vocabulary):
                    if word == term:
                        if word not in wordsAdded.keys():
                            wordsAdded[word] = {}
                            wordsAdded[word]['fileNames'] = []
                            wordsAdded[word]['Term_id'] = []
        #                     wordsAdded[word]['filePaths'] = []

                        wordsAdded[word]['fileNames'] += [f.name]
                        wordsAdded[word]['Term_id'] += [i]


    os.chdir(cwd)
    return wordsAdded
def writeToFile(words):
    with open('index-file.csv', 'w',encoding='UTF-8') as indexFile:

        fieldNames = ['Term_id','word', 'fileNames']

        csvWriter = csv.DictWriter(indexFile, fieldnames= fieldNames)

        csvWriter.writeheader()

        for word, fileDetails in words.items():

            fileNameString = reduce(lambda x, y: x + ", " + y, fileDetails['fileNames'])
            fileTerm_id = reduce(lambda  x, y: x, fileDetails['Term_id'])

            csvWriter.writerow({'word': word, 'fileNames': fileNameString, 'Term_id': fileTerm_id})


def main():
    directory = 'C:\\Users\\Profosor\\Desktop\\tsv'
    writeToFile(createDictionary(directory))
    print("Finished")


main()


Finished


In [13]:
import pandas as pd
df = pd.read_csv('index-file.csv')

In [18]:
df.tail(15)

,Term_id,word,fileNames
133,126,near,article_2.tsv
134,136,mage,article_2.tsv
135,144,stephani,article_2.tsv
136,154,dead,article_3.tsv
137,159,powel,article_3.tsv
138,163,stephen,article_3.tsv
139,151,asphyx,article_3.tsv
140,158,newbrook,article_3.tsv
141,172,assassin,article_4.tsv
142,177,windsor,article_4.tsv


In [16]:
df2 = df.copy()
df2.drop('word',axis=1,inplace=True)
t_id = df2['Term_id']
f_name = df2['fileNames']
d= dict(zip(t_id,f_name))
d

{0: 'article_0.tsv',
 2: 'article_0.tsv, article_0.tsv, article_0.tsv, article_1.tsv, article_1.tsv, article_1.tsv, article_3.tsv, article_3.tsv, article_3.tsv',
 3: 'article_0.tsv',
 8: 'article_0.tsv',
 9: 'article_0.tsv',
 12: 'article_0.tsv, article_0.tsv, article_1.tsv, article_1.tsv',
 14: 'article_0.tsv',
 17: 'article_0.tsv',
 21: 'article_0.tsv',
 23: 'article_0.tsv, article_0.tsv, article_2.tsv, article_2.tsv',
 24: 'article_0.tsv',
 170: 'article_0.tsv, article_4.tsv',
 32: 'article_0.tsv',
 33: 'article_0.tsv, article_4.tsv',
 39: 'article_0.tsv',
 160: 'article_0.tsv, article_2.tsv, article_3.tsv',
 45: 'article_0.tsv',
 5: 'article_0.tsv',
 6: 'article_0.tsv, article_0.tsv, article_3.tsv, article_3.tsv',
 10: 'article_0.tsv',
 15: 'article_0.tsv',
 16: 'article_0.tsv, article_0.tsv, article_1.tsv, article_1.tsv',
 18: 'article_0.tsv, article_0.tsv, article_1.tsv, article_1.tsv',
 20: 'article_0.tsv, article_0.tsv, article_0.tsv, article_0.tsv, article_0.tsv, article_1.tsv

In [17]:
for l,t in enumerate(vocabulary):
    print(l,t)

0 children
1 scienc
2 1972
3 zero
4 campu
5 almost
6 fiction
7 inspir
8 overpopul
9 bomb
10 popul
11 film
12 whose
13 danish
14 oliv
15 chaplin
16 design
17 ehrlich
18 concern
19 reflect
20 direct
21 bound
22 michael
23 best
24 earth
25 entir
26 american
27 featur
28 paul
29 short
30 dystopian
31 growth
32 sell
33 govern
34 denmark
35 geraldin
36 year
37 star
38 oppress
39 book
40 futur
41 reed
42 bleak
43 violat
44 world
45 execut
46 edna
47 marriag
48 shot
49 1972
50 pictur
51 woman
52 song
53 seven
54 susannah
55 screenplay
56 hutton
57 releas
58 mullin
59 theme
60 film
61 whose
62 london
63 british
64 upon
65 brien
66 separ
67 fool
68 last
69 design
70 singl
71 concern
72 taylor
73 direct
74 elizabeth
75 michael
76 middl
77 bicker
78 side
79 three
80 locat
81 director
82 need
83 known
84 columbia
85 album
86 cover
87 drama
88 cain
89 come
90 circl
91 well
92 famili
93 citat
94 coupl
95 base
96 brian
97 star
98 shepperton
99 novel
100 studio
101 night
102 compani
103 york
104 four
1